# First assessed lab - Question

Build a classifier for the MNIST dataset that achieves over 97% accuracy on the test set.
Use the first 60000 data samples for the training set and the remaining samples for the test set.
Hint: the *KNeighborsClassifier* works quite well for this task; you just need to find good hyperparameter values (try a grid search on the *weights* and *n_neighbors* hyperparameters).
Show the best hyperparameter configuration obtained via cross-validation and the best validation score. Finally, test the best model on the test set.

In [46]:
%matplotlib inline

from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist["data"], mnist["target"]

/Users/owenostermann/Desktop/Python_Enviroment/owenvenv/lib/python3.9/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Now that we have the data in a desirable form that is easy to work with. We have the 28X28 pixels, or the 784 columns of pixels and the 1 target column. Within the first 784 columns we have values between 0 and 255, 0 being a white background and 255 being a completely black background in the pixel. We also have the target, or the actual number that the pixels compose. As per the questions requirments, we are to train the model on the first 60,000 units and then test it on the last 10,000 units. Let's create our training and testing sets. 

In [47]:
# up to but not including, so here we get the first 60,000 to train
# from index 0 to 59,999
X_train, Y_train = X[0:60000], y[0:60000]

# Now we do the same for the testing set for x and y
X_test, Y_test = X[60000:70000], y[60000:70000]

# double check the lengths of the testing and training to ensure proper split
print("Length of Training Data: ", len(X_train))
print("Length of the Testing Data: ", len(X_test))

Length of Training Data:  60000
Length of the Testing Data:  10000


Now that we have set up the training and testing set we are ready to take our data and train it on the KNeighborsClassifier from sklearn. We will need to use a gridsearch to hyptertune for the weights and number of neighbours parameteres. Once we achive this, we will show the best hyperparameter configuration obtained via cross-validation and the best validation score.

First lets use a naive approach and use square root of the number of classes as K and then see which model preforms better between weights of distance and uniform. We will then optimize these parameteres later. 

In [48]:
from sklearn.neighbors import KNeighborsClassifier # importing the model from sklearn
import math
from sklearn.model_selection import cross_val_predict

# we choose our upper bound as the squre root of the number of predictors (784 pixel values)
# but since this is even, we want k + 1
k = int(math.sqrt(len(X[0])) + 1) 

# the two weight types we will try
weights_type = ["distance", "uniform"]

# now we are ready to train both of our models and test the accuracy to get a baseline
baseline_dis = KNeighborsClassifier(n_neighbors= k, weights = weights_type[0])
baseline_uni = KNeighborsClassifier(n_neighbors = k, weights = weights_type[1])
baseline_dis.fit(X_train, Y_train)
baseline_uni.fit(X_train, Y_train)

# corss validation predict 3 fold
y_train_pred_dis = cross_val_predict(baseline_dis, X_train, Y_train, cv=3)
y_train_pred_uni = cross_val_predict(baseline_uni, X_train, Y_train, cv=3)

In [51]:
from sklearn.metrics import accuracy_score

acc_dis_model = accuracy_score(Y_train, y_train_pred_dis)
acc_uni_model = accuracy_score(Y_train, y_train_pred_uni)

print("Accuracy of baseline on training set (k = 29, weight = distance) = ", acc_dis_model)
print("Accuracy of baseline on training set (k = 29, weight = uniform) = ", acc_uni_model)

Accuracy of baseline on training set (k = 29, weight = distance) =  0.9530166666666666
Accuracy of baseline on training set (k = 29, weight = uniform) =  0.9517166666666667


Now that we have these baselines, we can see that they are 95% ish accurate. But we want to do better. Lets use a gridsearch and cross validation method to hypertune the parameters to see if we can get our model to be even more accurate.

In [56]:
from sklearn.model_selection import GridSearchCV # importing the grid search/ cross validation

K_neigh = KNeighborsClassifier()
parameters = {'n_neighbors': [3, 5, 7], 'weights': ["distance", "uniform"]}
grid_search = GridSearchCV(estimator= K_neigh, param_grid = parameters, cv = 5)
grid_search.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7],
                         'weights': ['distance', 'uniform']})

In [61]:
print("Hypertuned K =", grid_search.best_params_["n_neighbors"])
print("Hypertuned Weights =", grid_search.best_params_["weights"])

print("Best Cross Validation Score: ", grid_search.best_score_)

Hypertuned K = 3
Hypertuned Weights = distance
Best Cross Validation Score:  0.9711166666666665


In [63]:
# get the predicted using the best model
y_predicted_best_model = grid_search.best_estimator_.predict(X_test)

In [65]:
# now evaluate the accuracy
best_model_accuracy = accuracy_score(y_predicted_best_model, Y_test)
print("Best Model Accuracy: ", best_model_accuracy)

Best Model Accuracy:  0.9717


So now we can see that the accuracy of our model has gone up from around 95% for the baselines up to 97.17% for the best model. We got a cross validation score of 97.11% and used the parameters K = 3 and weight = "distance". The model is successfull at predicting the handwritten digits in the MNIST dataset based on the provided pixels.   